In [1]:
from alefuncs import *

In [ ]:
#what is the weight in nanograms of a human genome? #data based on https://www.ncbi.nlm.nih.gov/genome/51
diploid_human_genome_mw = 6_469.66e6 * 607.4 #lenght (2n) * average weight of nucleotides
                                             #M.W. of dsDNA = (# nucleotides x 607.4)
                                             #https://www.thermofisher.com/us/en/home/references/ambion-tech-support/rna-tools-and-calculators/dna-and-rna-molecular-weights-and-conversions.html
hg_2n_mass_ng = convert_mw(diploid_human_genome_mw, to="ng")
hg_2n_mass_ng

In [ ]:
aploid_human_genome_mw = 3_234.83e6 * 607.4 #lenght (2n) * average weight of nucleotides
                                             #M.W. of dsDNA = (# nucleotides x 607.4)
                                             #https://www.thermofisher.com/us/en/home/references/ambion-tech-support/rna-tools-and-calculators/dna-and-rna-molecular-weights-and-conversions.html
hg_1n_mass_ng = convert_mw(aploid_human_genome_mw, to="ng")
hg_1n_mass_ng

In [ ]:
#how many genomes in 1ng of DNA?
hg_2n_per_ng = 1/hg_2n_mass_ng
hg_2n_per_ng

In [ ]:
hg_1n_Mb = 3_234.83 #Mb (Mega-basepairs) per haploid genome
hg_2n_Mb = 6_469.66 #Mb total (diploid).
hg_2n_mass_pg = hg_2n_Mb / 978 # 1 pg = 978 Mbp https://en.wikipedia.org/wiki/C-value
hg_2n_mass_pg

In [ ]:
1/(hg_2n_mass_pg/10**3)

In [ ]:
percent_of(5000,1)

In [ ]:
l = [0]*10

In [ ]:
%%timeit

split_overlap(l,7,2)


In [ ]:
from itertools import islice, tee, zip_longest    

nwise_overlap = lambda xs,n=2,lap=1,fill=None: zip_longest(*(islice(xs,idx,None,n-lap) for idx,xs in enumerate(tee(xs,n))), fillvalue=fill)

In [ ]:
l = [n for n in range(10)]
print(list(nwise_overlap(l,7,2)))
print(split_overlap(l,7,2))

In [ ]:
split_overlap(l,7,2)

In [2]:
from collections import deque

def split_overlap_jdehesa(iterable, size, overlap=0):
    size = int(size)
    overlap = int(overlap)
    if size < 1 or overlap < 0 or overlap >= size:
        raise ValueError()
    pops = size - overlap
    q = deque(maxlen=size)
    for elem in iterable:
        q.append(elem)
        if len(q) == size:
            yield tuple(q)
            for _ in range(pops):
                q.popleft()
    if len(q) > overlap:
        yield tuple(q)

In [3]:
import itertools

def split_overlap_Aviad(iterable, size, overlap):
    '''(iter,int,int) => [[...],[...],...]
    Split an iterable into chunks of a specific size and overlap.
    Works also on strings! 

    Examples:
        >>> split_overlap(iterable=list(range(10)),size=3,overlap=2)
        [[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9]]

        >>> split_overlap(iterable=range(10),size=3,overlap=2)
        [range(0, 3), range(1, 4), range(2, 5), range(3, 6), range(4, 7), range(5, 8), range(6, 9), range(7, 10)]
    '''
    if size < 1 or overlap < 0:
        raise ValueError('"size" must be an integer with >= 1 while "overlap" must be >= 0')
    result = []
    for i in zip(*[iterable[i::size-overlap] for i in range(size)]):
        result.append(i)
    return result

In [4]:
from itertools import zip_longest as iterzip

def split_overlap_Moinuddin(seq, size, overlap):
    for x in iterzip(*[seq[i::size-overlap] for i in range(size)]):
        yield [i for i in x if i!=None] 

In [54]:
from itertools import islice, tee, zip_longest

def nwise_overlap(xs, n=2, lap=1, fill=None):
    return zip_longest(*(islice(xs, idx, None, n - lap)
                         for idx, xs
                         in enumerate(tee(xs, n))
                         ))

In [70]:
def itr_split_overlap(iterable, size, overlap):
    itr = iter(iterable)
    buf = deque(islice(itr, size), maxlen=size)

    chunk = None
    for chunk in iter(lambda: tuple(islice(itr, size - overlap)), ()):
        yield tuple(buf)
        buf.extend(chunk)

    rest = tuple(buf)

    if chunk:
        rest = rest[size - overlap - len(chunk):]

    yield rest
    
def gen_split_overlap(seq, size, overlap):        
    if size < 1 or overlap < 0:
        raise ValueError('size must be >= 1 and overlap >= 0')

    for i in range(0, len(seq) - overlap, size - overlap):            
        yield seq[i:i + size]

In [100]:
def split_overlap_long(seq, size, overlap):
    '''(seq,int,int) => generator
    Split a sequence into chunks of a specific size and overlap.
    Return a generator. It is very efficient for long sequences (len(seq()) > 100).
    https://stackoverflow.com/questions/48381870/a-better-way-to-split-a-sequence-in-chunks-with-overlaps

    Examples:
        >>> split_overlap(seq=list(range(10)),size=3,overlap=2)
        [[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9]]

        >>> split_overlap(seq=range(10),size=3,overlap=2)
        [range(0, 3), range(1, 4), range(2, 5), range(3, 6), range(4, 7), range(5, 8), range(6, 9), range(7, 10)]
    '''       
    if size < 1 or overlap < 0:
        raise ValueError('size must be >= 1 and overlap >= 0')

    for i in range(0, len(seq) - overlap, size - overlap):            
        yield seq[i:i + size]

In [106]:
list(itr_split_overlap(iterable=list(range(10)),size=3,overlap=2))

[(0, 1, 2),
 (1, 2, 3),
 (2, 3, 4),
 (3, 4, 5),
 (4, 5, 6),
 (5, 6, 7),
 (6, 7, 8),
 (7, 8, 9)]

In [99]:
l = range(10)
s = 3
o = 2
print(split_overlap(l,s,o))
print(list(split_overlap_jdehesa(l,s,o)))
print(list(nwise_overlap(l,s,o)))
print(list(split_overlap_Moinuddin(l,s,o)))
print(list(gen_split_overlap(l,s,o)))
print(list(itr_split_overlap(l,s,o)))

[range(0, 3), range(1, 4), range(2, 5), range(3, 6), range(4, 7), range(5, 8), range(6, 9), range(7, 10)]
[(0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (7, 8, 9)]
[(0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (7, 8, 9), (8, 9, None), (9, None, None)]
[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9], [9]]
[range(0, 3), range(1, 4), range(2, 5), range(3, 6), range(4, 7), range(5, 8), range(6, 9), range(7, 10)]
[(0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (7, 8, 9)]


In [89]:
l = [str(n) for n in range(1000)]

In [90]:
%%timeit
split_overlap(l,7,2)

382 µs ± 4.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [91]:
%%timeit
list(split_overlap_jdehesa(l,7,2))

304 µs ± 5.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [92]:
%%timeit
list(nwise_overlap(l,7,2))

60 µs ± 967 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [93]:
%%timeit
list(split_overlap_Moinuddin(l,7,2))

157 µs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [94]:
%%timeit
list(gen_split_overlap(l,7,2))

44.2 µs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [95]:
%%timeit
list(itr_split_overlap(l,7,2))

184 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
